# <center> <font style="color:rgb(100,109,254)">  Car detection with Background Subtraction  </font> </center>

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

Remember previously we tried to do car detection with a Haar cascades and the results were not that good, so what we're gonna do now is try another approach, which is background subtraction & Contour detection since the video was recorded with a fixed camera and the cars are moving on a static background, we can achieve much greater results with this approach.

Our Whole Pipeline consists of 4 simple steps:

- **Step 1:** Extract cars with Background Subtraction and get rid of the shadows
- **Step 2:** Apply morphological operations to improve the backsub mask
- **Step 3:** Detect Contours and make sure you're not detecting noise
- **Step 4:** Compute a bounding box over the contour, draw it and show the image

In [3]:
# initlize video capture object
cap = cv2.VideoCapture('media/M4/carsvid.wmv')

# you can set custom kernel size if you want
kernel= None

# initilize background subtractor object
foog = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()
    if not ret:
        break
        
    # apply background subtraction
    fgmask = foog.apply(frame)
    
    # get rid of the shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    
    # apply some morphological operations to make sure you have a good mask
    fgmask = cv2.erode(fgmask,kernel,iterations = 1)
    fgmask = cv2.dilate(fgmask,kernel,iterations = 2)
    
    # Detect contours in the frame
    contours, hierarchy = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # loop over each contour in a given frame
    for cnt in contours:
        
        # make sure the contour area is somewhat hihger than some threshold to make sure its a car and not some noise.
        if cv2.contourArea(cnt) > 400:
            
            # Draw a bounding box around the car and labet it as car detected
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x ,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(frame,'Car Detected',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)

 
    # Stack all both frames and show the image
    fgmask_3 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    stacked = np.hstack((fgmask_3,frame))
    cv2.imshow('All three',cv2.resize(stacked,None,fx=0.65,fy=0.65))

    k = cv2.waitKey(1) & 0xff
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

 **`Note:` you can further improve this by experimenting with different background subtraction functions and threshold values, morphological params etc.**